In [2]:
%load_ext autoreload
%autoreload 2

from cnn import CNN
from dyn_cnn import DynCNN
from util import Util
from patch import Patch
from fixer import Fixer
from uglify import Uglify

import os
import mahotas as mh
import numpy as np
import time

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
%matplotlib inline

Using gpu device 0: GeForce GTX TITAN (CNMeM is disabled)
tifffile.py:1995: UserWarning: failed to import _tifffile.decodepackbits
  warnings.warn("failed to import %s" % module_function)
tifffile.py:1995: UserWarning: failed to import _tifffile.decodelzw
  warnings.warn("failed to import %s" % module_function)
tifffile.py:1995: UserWarning: failed to import _tifffile.unpackints
  warnings.warn("failed to import %s" % module_function)


In [3]:
#
# viren's approach with our LARGER border_overlap
#   image prob merged_array dyn_obj dyn_bnd border_overlap
networks = []
networks.append(CNN('viren_7', 'patches_7th', ['image', 'prob', 'merged_array']))
# networks.append(CNN('mine_7', 'patches_7th', ['image', 'prob', 'binary', 'border_overlap']))
# networks.append(CNN('mine_large_7', 'patches_7th', ['image', 'prob', 'binary', 'larger_border_overlap']))
networks.append(CNN('mine_merged_7', 'patches_7th', ['image', 'prob', 'merged_array', 'border_overlap']))
networks.append(CNN('mine_merged_large_7', 'patches_7th', ['image', 'prob', 'merged_array', 'larger_border_overlap']))
# networks.append(DynCNN('dyn_viren_7', 'patches_7th', ['image', 'prob', 'merged_array']))
# networks.append(DynCNN('dyn_mine_7', 'patches_7th', ['image', 'prob', 'binary', 'border_overlap']))
# networks.append(DynCNN('dyn_mine_large_7', 'patches_7th', ['image', 'prob', 'binary', 'larger_border_overlap']))
# networks.append(DynCNN('dyn_mine_merged_7', 'patches_7th', ['image', 'prob', 'merged_array', 'border_overlap']))
# networks.append(DynCNN('dyn_mine_merged_large_7', 'patches_7th', ['image', 'prob', 'merged_array', 'larger_border_overlap']))


Loading ['/Volumes/DATA1/split_cnn/patches_7th/viren_7/configuration.txt']
Loading /Volumes/DATA1/split_cnn/patches_7th/viren_7/network_350.p
Loading ['/Volumes/DATA1/split_cnn/patches_7th/mine_merged_7/configuration.txt']
Loading /Volumes/DATA1/split_cnn/patches_7th/mine_merged_7/network_251.p
Loading ['/Volumes/DATA1/split_cnn/patches_7th/mine_merged_large_7/configuration.txt']
Loading /Volumes/DATA1/split_cnn/patches_7th/mine_merged_large_7/network_262.p


In [4]:
DOJO_USER_DATA_FOLDER = '/Users/d/Dropbox/DOJOSTUDY/'
DOJO_USERS_IDS = [11,13,18,37,40,43,5,50,53,62]
DOJO_USERS = []
for d in DOJO_USERS_IDS:
    DOJO_USERS.append('S'+str(d)+'tif')
    
def read_user(user, slice):
    img = mh.imread(DOJO_USER_DATA_FOLDER+user+os.sep+'z='+str(slice).zfill(8)+'.tif')
    img = Util.crop_by_bbox(img, dojo_bbox)
    return Util.frame_image(img)

def dojo_user_vi(user, slice, gt):
    img = read_user(user, slice)
    return Util.vi(img, gt)

def fix_dojo_slice(cnn, slice, input_image, input_prob, input_rhoana, input_gold,t=0.):
    DOJO_SLICE=slice
    vi_s, merge_pairs, surenesses = Fixer.splits(cnn, 
                                                 input_image[DOJO_SLICE], 
                                                 input_prob[DOJO_SLICE], 
                                                 input_rhoana[DOJO_SLICE], 
                                                 input_gold[DOJO_SLICE],
                                                 sureness_threshold=t,
                                                 oversampling=False)

    best_index = vi_s.index(np.min(vi_s))
    best_vi = vi_s[best_index]
    best_sureness = surenesses[best_index]
    
    return best_vi, best_sureness, vi_s, surenesses

def fix_dojo_slice_user_simulated(cnn, slice, input_image, input_prob, input_rhoana, input_gold, error_rate=0,t=0):
    DOJO_SLICE=slice
    vi_s, merge_pairs, surenesses, good_fixes, bad_fixes = Fixer.splits_user_simulated(cnn, 
                                                 input_image[DOJO_SLICE], 
                                                 input_prob[DOJO_SLICE], 
                                                 input_rhoana[DOJO_SLICE], 
                                                 input_gold[DOJO_SLICE],
                                                 error_rate=error_rate,
                                                 sureness_threshold=t,                                                 
                                                 oversampling=False)
    
    best_index = vi_s.index(np.min(vi_s))
    best_vi = vi_s[best_index]
    best_sureness = surenesses[best_index]
    
    return best_vi, good_fixes, bad_fixes, merge_pairs, vi_s

def threed_vi(gt, seg):
    total_vi = 0
    slice_vi = []    
    for i in range(10):
        current_vi = Util.vi(gt[i].astype(np.int64), seg[i].astype(np.int64))
        total_vi += current_vi
        slice_vi.append(current_vi)
    total_vi /= 10
    return total_vi, slice_vi

def user_threed_vi(gt, user):
    total_vi = 0
    slice_vi = []
    for i in range(10):
        current_vi = dojo_user_vi(user, i, gt[i])
        total_vi += current_vi
        slice_vi.append(current_vi)
    total_vi /= 10
    return total_vi, slice_vi
    
def cnn_threed_vi(cnn, input_image, input_prob, input_rhoana, input_gold):
    total_vi = 0
    slice_vi = []
    slice_sureness = []
    for i in range(10):
        current_vi, sureness = fix_dojo_slice(cnn, i, input_image, input_prob, input_rhoana, input_gold)
        total_vi += current_vi
        slice_vi.append(current_vi)
        slice_sureness.append(sureness)
    total_vi /= 10
    return total_vi, slice_vi, slice_sureness
        

In [5]:
input_image, input_prob, input_gold, input_rhoana, dojo_bbox = Util.read_dojo_data() 

In [44]:
simulated_user_er_vi_s =[]

for er in range(0,20,2):
    
    er = er/100.

#     simulated_user_vi_s2 = []
    bigM, out_volume = Fixer.splits_global(networks[2], input_image, input_prob, input_rhoana, input_gold, error_rate=er)
        
    simulated_user_er_vi_s.append(threed_vi(input_gold, out_volume)[1])

464 generated in 10.2781190872 seconds.
Grouped into 116 patches in 0.0029821395874 seconds.
448 generated in 12.3324241638 seconds.
Grouped into 112 patches in 0.00846910476685 seconds.
456 generated in 12.370153904 seconds.
Grouped into 113 patches in 0.00935101509094 seconds.
484 generated in 12.7441439629 seconds.
Grouped into 121 patches in 0.0144698619843 seconds.
508 generated in 13.2544138432 seconds.
Grouped into 127 patches in 0.0146548748016 seconds.
488 generated in 12.930325985 seconds.
Grouped into 122 patches in 0.0142619609833 seconds.
480 generated in 12.6225888729 seconds.
Grouped into 118 patches in 0.0131599903107 seconds.
460 generated in 12.3240158558 seconds.
Grouped into 113 patches in 0.0144739151001 seconds.
488 generated in 12.7894229889 seconds.
Grouped into 120 patches in 0.0124979019165 seconds.
540 generated in 14.1270699501 seconds.
Grouped into 135 patches in 0.0176601409912 seconds.
464 generated in 10.2133538723 seconds.
Grouped into 116 patches in 0.

In [40]:
import random
random.random()

0.28629667154543414

In [32]:
for er in range(0,20,2):
    
    er = er/100.
    print er

0.0
0.02
0.04
0.06
0.08
0.1
0.12
0.14
0.16
0.18


In [45]:
simulated_user_er_vi_s

[[0.4050252609261369,
  0.4121594568115894,
  0.3918172770092099,
  0.4053514521002537,
  0.4337343707604937,
  0.41851464530707894,
  0.5240820744039167,
  0.5830389613197333,
  0.6172253790488904,
  0.4922499542447536],
 [0.4050252609261369,
  0.4121594568115894,
  0.3918172770092099,
  0.4053514521002537,
  0.4448077191645332,
  0.4811246236263482,
  0.5240820744039167,
  0.5830389613197333,
  0.6172253790488904,
  0.4922499542447536],
 [0.4050252609261369,
  0.4121594568115894,
  0.3918172770092099,
  0.4053514521002537,
  0.4448077191645332,
  0.45510110571296725,
  0.5240820744039167,
  0.5830389613197333,
  0.6172253790488904,
  0.5093931380673915],
 [0.4050252609261369,
  0.4121594568115894,
  0.3931952501947853,
  0.4085899806629616,
  0.4448077191645332,
  0.4652438556229148,
  0.5299399791105088,
  0.5830389613197333,
  0.6172253790488904,
  0.4922499542447536],
 [0.4050252609261369,
  0.4121594568115894,
  0.3918172770092099,
  0.4053514521002537,
  0.4448077191645332,
  0.

In [48]:
out =[]
for s in simulated_user_er_vi_s:
    out.append(np.median(s))

In [49]:
out

[0.42612450803378632,
 0.46296617139544072,
 0.44995441243875023,
 0.45502578739372401,
 0.43752289166950975,
 0.45261827858947168,
 0.46369569416586565,
 0.45728723762254075,
 0.47940199025731012,
 0.48589741139750675]

In [7]:
bigM = Fixer.splits_global(networks[2], input_image, input_prob, input_rhoana, input_gold)

464 generated in 10.3521120548 seconds.
Grouped into 116 patches in 0.0029411315918 seconds.
448 generated in 12.3641450405 seconds.
Grouped into 112 patches in 0.00849795341492 seconds.
456 generated in 12.471585989 seconds.
Grouped into 113 patches in 0.00862312316895 seconds.
484 generated in 12.9273130894 seconds.
Grouped into 121 patches in 0.00880193710327 seconds.
508 generated in 13.5673720837 seconds.
Grouped into 127 patches in 0.0124070644379 seconds.
488 generated in 13.3480451107 seconds.
Grouped into 122 patches in 0.0130300521851 seconds.
480 generated in 12.7625470161 seconds.
Grouped into 118 patches in 0.0167269706726 seconds.
460 generated in 12.4207539558 seconds.
Grouped into 113 patches in 0.0106129646301 seconds.
488 generated in 12.8981289864 seconds.
Grouped into 120 patches in 0.0126349925995 seconds.
540 generated in 14.1090538502 seconds.
Grouped into 135 patches in 0.0108759403229 seconds.


In [8]:
import cPickle as pickle
with (open('/tmp/m.npy', 'rb')) as f:
    merge_fixes = pickle.load(f)

In [13]:
all_splits = []
for m in bigM:
    all_splits.extend (np.tril(m).flatten())

In [17]:
all_merges = []
for m in merge_fixes:
    pred = 1-m[2]
    all_merges.append(pred)

In [1]:
bigM

NameError: name 'bigM' is not defined

In [16]:
max(all_splits)

0.99518787860870361

In [19]:
sorted(all_merges)

[0.0063292980194091797,
 0.0063292980194091797,
 0.0063292980194091797,
 0.0063292980194091797,
 0.0063292980194091797,
 0.0063292980194091797,
 0.013832509517669678,
 0.013832509517669678,
 0.013832509517669678,
 0.013832509517669678,
 0.013832509517669678,
 0.013832509517669678,
 0.015900135040283203,
 0.015900135040283203,
 0.015900135040283203,
 0.015900135040283203,
 0.015900135040283203,
 0.015900135040283203,
 0.017456531524658203,
 0.017456531524658203,
 0.017456531524658203,
 0.017456531524658203,
 0.017456531524658203,
 0.017456531524658203,
 0.019558429718017578,
 0.019558429718017578,
 0.019914686679840088,
 0.019914686679840088,
 0.019914686679840088,
 0.019914686679840088,
 0.019914686679840088,
 0.020681560039520264,
 0.020681560039520264,
 0.020681560039520264,
 0.020681560039520264,
 0.020681560039520264,
 0.020681560039520264,
 0.021589756011962891,
 0.021589756011962891,
 0.021589756011962891,
 0.021589756011962891,
 0.021589756011962891,
 0.021589756011962891,
 0.02

In [22]:
sorted(all_splits)[-120]

0.9241829514503479

In [23]:
mat, out_vol = Fixer.splits_global(networks[2], input_image, input_prob, input_rhoana, input_gold)

464 generated in 10.6337718964 seconds.
Grouped into 116 patches in 0.00292205810547 seconds.
448 generated in 12.4978489876 seconds.
Grouped into 112 patches in 0.00987482070923 seconds.
456 generated in 12.6088337898 seconds.
Grouped into 113 patches in 0.00814008712769 seconds.
484 generated in 12.9472639561 seconds.
Grouped into 121 patches in 0.012130022049 seconds.
508 generated in 13.5443570614 seconds.
Grouped into 127 patches in 0.0100979804993 seconds.
488 generated in 13.5390720367 seconds.
Grouped into 122 patches in 0.0110239982605 seconds.
480 generated in 12.8854839802 seconds.
Grouped into 118 patches in 0.00895810127258 seconds.
460 generated in 12.3480100632 seconds.
Grouped into 113 patches in 0.0100660324097 seconds.
488 generated in 12.8427321911 seconds.
Grouped into 120 patches in 0.00880408287048 seconds.
540 generated in 14.1459290981 seconds.
Grouped into 135 patches in 0.0119431018829 seconds.
